In [1]:
import pandas as pd
import numpy as np

## Output
1. adjust graularity
2. calulate publication count
3. filter by non-zero centrality

In [16]:
df = pd.read_csv('EJOR_vol_256-295.csv', index_col=False)
df.head()

# Adjust Graularity
def aff2univ(str):
    univ = [x for x in str.split(', ') if x.lower().find('univ') !=-1]
    if len(univ) > 0:
        univ = univ[0]
    else:
        univ = str
    return univ
df['affiliation'] = df['affiliation'].apply(lambda s: aff2univ(s))
df.head()

# Calculate weight for publications
df['pub'] = df.apply(lambda x: 1, axis=1)
df['wtd pub'] = df['author idx'].apply(lambda x: 2 if x==1 else 1)
art_cnt = df['title'].value_counts().to_dict()
df['auth cnt'] = df['title'].apply(lambda name: art_cnt[name])
df['wtd pub'] = df.apply(lambda x: x['wtd pub'] / (x['auth cnt']+1), axis=1)
df.head()

# Calculate publications of each institue
non_zero_list = np.load('institute_list_not_zero_uni.npy')
inst = pd.DataFrame({'institute': [x for x in df['affiliation'].unique() if x in non_zero_list]})
inst['publication'] = inst['institute'].apply(lambda name: df.loc[df['affiliation']==name]['pub'].sum())
inst['weighted publication'] = inst['institute'].apply(lambda name: df.loc[df['affiliation']==name]['wtd pub'].sum())
inst['log pub'] = inst['publication'].apply(lambda x: np.log(x))
inst['log wtd pub'] = inst['weighted publication'].apply(lambda x: np.log(x))
inst.to_csv('output-non-zero.csv')
inst.sort_values(by=['publication'], ascending=False)

,institute,publication,weighted publication,log pub,log wtd pub
61,Ghent University,54,21.183333,3.988984,3.053215
437,Tianjin University,48,14.083333,3.871201,2.644992
133,University of Southampton,30,9.211905,3.401197,2.220497
140,Technical University of Denmark,27,7.966667,3.295837,2.075266
22,Eindhoven University of Technology,27,8.783333,3.295837,2.172856
...,...,...,...,...,...
800,University of Gävle,1,0.166667,0.000000,-1.791759
801,Yangzhou University,1,0.166667,0.000000,-1.791759
802,Guizhou Minzu University,1,0.166667,0.000000,-1.791759
805,University of Regina,1,0.200000,0.000000,-1.609438


## Input
1. adjust granularity
2. filter by degree centrality (non-zero)

In [9]:
deg = pd.read_csv('input.csv', index_col=False)
deg = deg[['institute', 'degree centrality in single-edge graph', 'degree centrality in multi-edge graph']]
deg = deg.rename(columns={'degree centrality in single-edge graph': 'degree centrality (single)', 'degree centrality in multi-edge graph': 'degree centrality (multi)'})

# Adjust degree centrality
deg['inst'] = deg['institute'].apply(lambda s: aff2univ(s))

# non-zero centrality
dfin = pd.DataFrame({'institute': deg['inst'].unique()})
dfin['degree centrality (single)'] = dfin['institute'].apply(lambda name: deg.loc[deg['institute']==name]['degree centrality (single)'].sum())
dfin['degree centrality (multi)'] = dfin['institute'].apply(lambda name: deg.loc[deg['institute']==name]['degree centrality (multi)'].sum())
dfin.loc[dfin['degree centrality (multi)'] > 0].to_csv('input-non-zero.csv', index=False)
non_zero = dfin.loc[dfin['degree centrality (multi)'] > 0]['institute'].to_list()